In [32]:
from bs4 import BeautifulSoup as bs
from requests import Response, Request, request
from requests.compat import urljoin, urlparse, quote, unquote
from urllib.parse import urlparse, parse_qs
import re
import requests

In [14]:
#헤더 값
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'
}

In [15]:
#방송사별 가장 첫 뉴스 주소
news_list={
    'KBS' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=355&date=20130613',
    'MBC' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=370&date=201300101',
    'SBS' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=371&date=201300101',
    'JTBC' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=742&date=20131014',
    'TVC' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=750&date=20131125',
    'YTN' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=751&date=20131121',
    'CHA' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=74b&date=20131125',
    'MBN' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=74e&date=20131125',
    'NESY' : 'https://news.naver.com/main/tvgallery/replay.naver?sid=74f&date=20140106',   
}

In [16]:
#페이지 get 방식으로 받기
for i,j in news_list.items():
    main_resp=request(url=j,headers=headers,method='GET')
    print(main_resp)
    break

<Response [200]>


In [17]:
main_page=bs(main_resp.text,"lxml")

In [18]:
main_page

<!DOCTYPE HTML>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="3600" http-equiv="refresh"/>
<meta content="width=1106" name="viewport"/>
<meta content="네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="http://news.naver.com/main/home.naver" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image"/>
<meta content="정치, 경제, 사회, 생활/문화, 세계, IT/과학 등 언론사별, 분야별 뉴스 기사 제공" property="og:description"/>
<meta content="네이버" property="og:article:author"/>
<meta content="summary" name="twitter:card"/>
<meta content="네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="twitter:image"/>
<meta conte

In [19]:
temp=main_page.find('iframe')
print(temp)
#문제 플레이어의 소스는 iframe 안에 숨겨져 있는데 iframe 안 src 주소가 보이질 않음 
#직접 주소를 따보아서 주소의 패턴을 알아보고자 함


<iframe allowfullscreen="true" frameborder="0" height="476" id="article_vod" marginheight="0" marginwidth="0" name="article_vod" src="" title="영상 플레이어" width="780"></iframe>


In [20]:
vid_num=main_page.find(attrs={'class':'nclicks(rep_kbs.timg) NCR-ASYNC'}).attrs['href']

In [21]:
vid_url="https://news.naver.com/main/readVod.naver?width=780&height=476&callbackHandler=parent.news.gallery.replay.GalleryManager.callback&oid={}&aid={}".format(vid_num[1:4],vid_num[5:])

In [22]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
    'Referer' : j # 매번 페이지가 바뀌니 페이지 주소를 받아와야 함 (레퍼러 없으면 안불러와짐)
}

In [23]:
vid_page=bs(request(url=vid_url,headers=headers,method='POST').text,'html5lib')

In [24]:
vid_page

<!DOCTYPE html>
<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<style type="text/css">
html,body{height:100%;overflow:hidden}
</style>
</head>
<body>
<script charset="UTF-8" src="https://sstatic-rmcnmv.pstatic.net/js/webplayer_4.18.40.all.26b7e2e3.min.js?dummy=20210722_171150" type="text/javascript"></script>
<script type="text/javascript">
document.domain='naver.com';


var rmcPlayer = new nhn.rmcnmv.RMCVideoPlayer('5BBBC4CAF42719A59C7E898781CF5D8C032D', {'value':'V124d9712c82c23ca27f8ba6b3d1a47100729ce7e3fcc954ee2b6a2dd0df556fb52f2ba6b3d1a47100729', 'outKeyMode':false}, 0, 0, '2006', null, "https://serviceapi.rmcnmv.naver.com");
rmcPlayer.setProperties({
	defaultResolution	: "720",
	hasRelativeMovie	: true,
	showContentInfo		: false,
	autoPlay			: true,
	coverImageURL		: "https://imgnews.pstatic.net/image/056/2013/06/13/2674840_20.jpg",
	wmode				: "opaque",
	skinName			: "tvcast_white",


In [25]:
vid_temp0=vid_page.find("div")

In [26]:
print(vid_temp0)

None


In [27]:
vid_temp1=vid_page.find_all(attrs={'type':'text/javascript'})[1]

In [28]:
vid_id=re.findall("(?<=\(\\\')([A-Za-z0-9_-]*)(?=\\\')",vid_temp1.text)[0]
vid_key=re.findall("(?<=\\\'value\\\'\:\\')([A-Za-z0-9_-]*)(?=\\\')",vid_temp1.text)[0]

In [29]:
vid_id,vid_key

('5BBBC4CAF42719A59C7E898781CF5D8C032D',
 'V124d9712c82c23ca27f8ba6b3d1a47100729ce7e3fcc954ee2b6a2dd0df556fb52f2ba6b3d1a47100729')

In [30]:
pid 값도 받아야 하는데 아직 못찾음 
rmcPlayer_16271713763554756

SyntaxError: invalid syntax (<ipython-input-30-68afb7a69d3c>, line 1)

In [31]:
vid_params={
    "key": vid_key,
    "pid": 'rmcPlayer_16271713763554756',
    "sid": "2006",
    "ver": "2.0",
    "devt": "html5_pc",
    "doct": "json",
    "ptc": "https",
    "sptc": 'https',
    "cpt": "vtt",
    "ctls": "%7B%22visible%22%3A%7B%22fullscreen%22%3Atrue%2C%22logo%22%3Afalse%2C%22playbackRate%22%3Afalse%2C%22scrap%22%3Afalse%2C%22playCount%22%3Atrue%2C%22commentCount%22%3Atrue%2C%22title%22%3Atrue%2C%22writer%22%3Atrue%2C%22expand%22%3Afalse%2C%22subtitles%22%3Atrue%2C%22thumbnails%22%3Atrue%2C%22quality%22%3Atrue%2C%22setting%22%3Atrue%2C%22script%22%3Afalse%2C%22logoDimmed%22%3Atrue%2C%22badge%22%3Atrue%2C%22seekingTime%22%3Atrue%2C%22muted%22%3Atrue%2C%22muteButton%22%3Afalse%2C%22viewerNotice%22%3Afalse%2C%22linkCount%22%3Afalse%2C%22createTime%22%3Afalse%2C%22thumbnail%22%3Atrue%7D%2C%22clicked%22%3A%7B%22expand%22%3Afalse%2C%22subtitles%22%3Afalse%7D%7D",
    "pv": "4.18.40",
    "dr": "2560x1440",
    "lc": "ko_KR",
    "videoId": vid_id
}

In [34]:
parse.urlparse("https://apis.naver.com/rmcnmv/rmcnmv/vod/play/v2.0/5BBBC4CAF42719A59C7E898781CF5D8C032D?key=V1289d1726102579fa6f2ba6b3d1a47100729ce7e3fcc954ee2b6a2dd0df556fb52f2ba6b3d1a47100729&pid=rmcPlayer_16271713763554756&sid=2006&ver=2.0&devt=html5_pc&doct=json&ptc=https&sptc=https&cpt=vtt&ctls=%7B%22visible%22%3A%7B%22fullscreen%22%3Atrue%2C%22logo%22%3Afalse%2C%22playbackRate%22%3Afalse%2C%22scrap%22%3Afalse%2C%22playCount%22%3Atrue%2C%22commentCount%22%3Atrue%2C%22title%22%3Atrue%2C%22writer%22%3Atrue%2C%22expand%22%3Afalse%2C%22subtitles%22%3Atrue%2C%22thumbnails%22%3Atrue%2C%22quality%22%3Atrue%2C%22setting%22%3Atrue%2C%22script%22%3Afalse%2C%22logoDimmed%22%3Atrue%2C%22badge%22%3Atrue%2C%22seekingTime%22%3Atrue%2C%22muted%22%3Atrue%2C%22muteButton%22%3Afalse%2C%22viewerNotice%22%3Afalse%2C%22linkCount%22%3Afalse%2C%22createTime%22%3Afalse%2C%22thumbnail%22%3Atrue%7D%2C%22clicked%22%3A%7B%22expand%22%3Afalse%2C%22subtitles%22%3Afalse%7D%7D&pv=4.18.40&dr=2560x1440&lc=ko_KR&videoId=5BBBC4CAF42719A59C7E898781CF5D8C032D")

ParseResult(scheme='https', netloc='apis.naver.com', path='/rmcnmv/rmcnmv/vod/play/v2.0/5BBBC4CAF42719A59C7E898781CF5D8C032D', params='', query='key=V1289d1726102579fa6f2ba6b3d1a47100729ce7e3fcc954ee2b6a2dd0df556fb52f2ba6b3d1a47100729&pid=rmcPlayer_16271713763554756&sid=2006&ver=2.0&devt=html5_pc&doct=json&ptc=https&sptc=https&cpt=vtt&ctls=%7B%22visible%22%3A%7B%22fullscreen%22%3Atrue%2C%22logo%22%3Afalse%2C%22playbackRate%22%3Afalse%2C%22scrap%22%3Afalse%2C%22playCount%22%3Atrue%2C%22commentCount%22%3Atrue%2C%22title%22%3Atrue%2C%22writer%22%3Atrue%2C%22expand%22%3Afalse%2C%22subtitles%22%3Atrue%2C%22thumbnails%22%3Atrue%2C%22quality%22%3Atrue%2C%22setting%22%3Atrue%2C%22script%22%3Afalse%2C%22logoDimmed%22%3Atrue%2C%22badge%22%3Atrue%2C%22seekingTime%22%3Atrue%2C%22muted%22%3Atrue%2C%22muteButton%22%3Afalse%2C%22viewerNotice%22%3Afalse%2C%22linkCount%22%3Afalse%2C%22createTime%22%3Afalse%2C%22thumbnail%22%3Atrue%7D%2C%22clicked%22%3A%7B%22expand%22%3Afalse%2C%22subtitles%22%3Afalse%7

In [8]:
headersT={
    "y": "news.naver.com",
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "ko-KR,ko;q=0.9",
    "charset": "utf-8",
    "content-length": "0",
    "content-type": "application/x-www-form-urlencoded; charset=utf-8",
    "cookie": "JSESSIONID=10DF963758A4D9F14BE0A437C2534AB5",
    "origin": "https://news.naver.com",
    "referer": "https://news.naver.com/main/tvgallery/replay.naver?sid=355&date=20130613",
    "sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.3'

}

In [9]:
test_page = bs(request(url='https://news.naver.com/main/tvgallery/item.naver?mode=undefined&oid=056&aid=0000003462',
                       headers=headersT, method='POST').text, 'html5lib')

In [11]:
test_page

<html><head></head><body>{oid:"056",
aid:"0000003462",
gdid:"88000114_000000000000000000003462",
type:"2",
title:"\uCF54\uC2A4\uD53C \uC5F0\uC911 \uCD5C\uC800\uCE58\u2026\uC544\uC2DC\uC544 \uC99D\uC2DC\uB3C4 \uD3ED\uB77D",
officeName:"KBS",
createDate:"2013-06-13 22:25",
modifyDate:"2013-06-13 22:25",
content:"&lt;\uC575\uCEE4 \uBA58\uD2B8&gt;  \uC2DC\uCCAD\uC790 \uC5EC\uB7EC\uBD84,  \uC548\uB155\uD558\uC2ED\uB2C8\uAE4C? \uC624\uB298 \uCF54\uC2A4\uD53C \uC9C0\uC218\uAC00 \uC62C \uB4E4\uC5B4 \uCD5C\uC800\uCE58\uB85C  \uB5A8\uC5B4\uC84C\uC2B5\uB2C8\uB2E4. \uC6B0\uB9AC \uC99D\uC2DC\uBFD0 \uC544\uB2C8\uB77C \uC77C\uBCF8\uACFC \uD64D\uCF69 \uB4F1 \uC544\uC2DC\uC544 \uC99D\uC2DC \uC804\uBC18\uC774 \uD070 \uD558\uB77D\uC138\uB97C \uBCF4\uC774\uACE0 \uC788\uC2B5\uB2C8\uB2E4. \uBA3C\uC800 \uBC15\uC77C\uC911 \uAE30\uC790\uAC00  \uBCF4\uB3C4\uD569\uB2C8\uB2E4.    &lt;\uB9AC\uD3EC\uD2B8&gt;  \uCF54\uC2A4\uD53C\uAC00  \uC0AC\uD758 \uC5F0\uC18D \uD558\uB77D\uD588\uC2B5\uB2C8\uB2E4.  \uB354\uC6B0\uAE